# Solutions PID Numérique

- Discrétiser un PI mixte

La fonction de transfert d'un PI mixte est la suivante:

$$ \frac{U(s)}{E(s)} = K_p \left( 1 + \frac{1}{T_i s} \right) $$

Afin de gérer l'intégrale de manière indépendante, discrétisons les actions séparemment. Le PI mixte sera alors calculé ainsi:

\begin{align}
  U_p(s) &= E(s) \\
  U_i(s) &= \frac{E(s)}{T_i s} \\
  U(s) &= K_p (U_p(s) + U_i(s))
\end{align}

La discrétisation de $U_p(s)$ donne:

$$ u_p(k) = e(k) $$

La discrétisation de $U_i(s)$ donne:

\begin{align}
  U_i(s) &= \frac{E(s)}{T_i s} \\
  T_i s U_i(s) &= E(s) \\
  T_i \frac{du_i(t)}{dt} &= e(t) \\
  T_i \frac{u_i(k) - u_i(k-1)}{h} &= e(k) \\
  u_i(k) &= \frac{h}{T_i}e(k) + u_i(k-1)
\end{align}

Le régulateur PI mixte devient alors:

\begin{align}
  u(k) &= K_p ( u_p(k) + u_i(k) ) \\
  &= K_p \left( e(k) + \frac{h}{T_i} e(k) + u_i(k-1) \right)
\end{align}

- Ecrire le pseudo code en gérant l'erreur d'intégration et l'anti-emballement

L'équation aux différences précédentes peut engendrer des erreurs d'intégrations. En développant le termine $u_i(k-1)$, l'intégrale devient:

\begin{align}
  u_i(k) &= \frac{h}{T_i}e(k) + u_i(k-1) \\
  u_i(k) &= \frac{h}{T_i}e(k) + \frac{h}{T_i}e(k-1) + u_i(k-2) \\
  u_i(k) &= \frac{h}{T_i}e(k) + \frac{h}{T_i}e(k-1) + \frac{h}{T_i}e(k-2) + u_i(k-3) \\
  \vdots \\
  u_i(k) &= \frac{h}{T_i}e(k) + \frac{h}{T_i}e(k-1) + \frac{h}{T_i}e(k-2) + \dots + \frac{h}{T_i}e(0)
\end{align}

En mettant $\frac{h}{T_i}$ en évidence, on obtient:

$$ u_i(k) = \frac{h}{T_i} ( e(k) + e(k-1) + e(k-2) + \dots + e(0) ) $$

Il est donc possible d'accumuler les erreurs dans une variable intermédiaire:

\begin{align}
  i(k) &= i(k-1) + e(k) \\
  u_i(k) &= \frac{h}{T_i} i(k)
\end{align}

Le pseudo code complet est alors:

\begin{align}
  &e(k) = sp(k) - pv(k) \\
  &i(k) = i(k-1) + e(k) \\
  &u_i(k) = \frac{h}{T_i} i(k) \\
  &u(k) = K_p (u_p(k) + u_i(k)) \\
  &\text{si}\, |u(k)| > \mu: \\
  &\quad i(k) = i(k-1) \\
  &\quad u_i(k) = \frac{h}{T_i} i(k) \\
  &\quad u(k) = K_p (u_p(k) + u_i(k))
\end{align}

- Programmer le régulateur sur Arduino

- Tester le régulateur